In [2]:
###Importing Modules

import matplotlib.pyplot as plt
import numpy as np
import h2o
import pandas as pd
import csv
import os.path

###Removing current instances of H2o and initialising H2o

h2o.init(ip='localhost', nthreads=10,
                     min_mem_size='1G', max_mem_size='8G')
h2o.remove_all()

###Importing H2o

from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.model.metrics_base import H2OBinomialModelMetrics 
from h2o.grid.metrics import H2OBinomialGridSearch
from h2o.grid.grid_search import H2OGridSearch
from h2o.model import H2OBinomialModel
from h2o.model.model_base import ModelBase

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_151"; OpenJDK Runtime Environment (build 1.8.0_151-8u151-b12-0ubuntu0.17.10.2-b12); OpenJDK 64-Bit Server VM (build 25.151-b12, mixed mode)
  Starting server from /home/will/Computerscience/Anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpj5_lb_rs
  JVM stdout: /tmp/tmpj5_lb_rs/h2o_will_started_from_python.out
  JVM stderr: /tmp/tmpj5_lb_rs/h2o_will_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/London
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_will_rhpt57
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,10
H2O cluster status:,"accepting new members, healthy"


In [4]:
###loading raw data
#training

training_data = h2o.import_file("/home/will/Computerscience/Machinelearning/Projects/Toxicspeechspot/Programdata/train_tagged.csv")

print("Training CSV file imported successfully")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Training CSV file imported successfully


In [5]:
###Splitting training data into training, validation and testing 

train, valid = training_data.split_frame([.7])

print("Frame split correctly")

Frame split correctly


In [6]:
#testing
testing_data = h2o.import_file("/home/will/Computerscience/Machinelearning/Projects/Toxicspeechspot/Programdata/test_tagged.csv")

print("Testing CSV file imported successfully")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Testing CSV file imported successfully


In [7]:
###Defining predictor and response
#train["comment_text","tagged_length"] = train["comment_text","tagged_length"].asfactor()
train["toxic"] = train["toxic"].asfactor()
predictor = ["comment_text","tagged_length"]
response = "toxic"

print("Predictor and repsonese assigned")

Predictor and repsonese assigned


In [8]:
###grid_search parameters

criteria = {'strategy': 'RandomDiscrete',
           'max_models':1000,
           'seed': 1234
           }

###Defining hyper params

sample_rate_hp = [i * 0.1 for i in range(1, 9)]
ntrees_hp = 10 #[100,500,1000, 2000, 3000, 4000]
folds = 5

hyperparams= {"sample_rate": sample_rate_hp , "ntrees" : ntrees_hp}

print("Hyper parameters and grid search parameters assigned")


Hyper parameters and grid search parameters assigned


In [9]:
###Define the grid search

toxic_rdf_grid = H2OGridSearch(model=H2ORandomForestEstimator,
                              grid_id='toxic_rdf_grid_id',
                              hyper_params=hyperparams,
                              search_criteria=criteria)

print("Grid search defined")

Grid search defined


In [10]:
###Running the grid search

toxic_rdf_grid.train(predictor,
                    response,
                    training_frame=train,
                    validation_frame=valid)#,
                     #nfolds = 5)

drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [11]:
###model performance sorted by logloss
model_perf = toxic_rdf_grid.sorted_metric_table() 

written = False
i = 0

while written == False:
    if os.path.exists("/home/will/Computerscience/Machinelearning/Projects/Toxicspeechspot/Modelperformance/model_performance{0}.csv".format(i)) == True:
        i = i + 1
    else:
        model_perf.to_csv("/home/will/Computerscience/Machinelearning/Projects/Toxicspeechspot/Modelperformance/model_performance{0}.csv".format(i))
        break